# Práctica Machine Learning

#### Autor: Ramón-Armand Laborda Moreu

El objetivo de la práctica consiste en la búsqueda de un modelo óptimo que pueda estimar el precio de una vivienda con la suficiente precisión cumpliendo las principales asunciones estadísticas. El dataset es el resultado de un scraping a la web de Airbnb. A grandes rasgos se pretende cubrir los siguientes puntos centrados en un problema de regresión.

1. Preparación de datos
2. Análisis exploratorio
3. Preprocesamiento de datos
4. Modelado
5. Conclusión

#### 1. Preparación de datos

In [2]:
import pandas as pd

# Cargamos fichero csv y almacenamos en pandas df
airbnb = pd.read_csv(r'data/airbnb-listings-extract.csv', sep=';')

Una vez cargado el dataset y previo a separar en dos muestras (train y test) visualizamos una muestra de los datos así como la estructura general para detectar cual es la variable dependiente (objeto de estudio) y las independientes. Si es necesario eliminamos variables que carezcan de información general u observaciones no etiquetadas de la variable dependiente.

In [5]:
airbnb.head(7)

,ID,Listing Url,Scrape ID,Last Scraped,Name,Summary,Space,Description,Experiences Offered,Neighborhood Overview,...,Review Scores Communication,Review Scores Location,Review Scores Value,License,Jurisdiction Names,Cancellation Policy,Calculated host listings count,Reviews per Month,Geolocation,Features
0,11210388,https://www.airbnb.com/rooms/11210388,20170306202425,2017-03-07,The Loft-Full Bath-Deck w/View,Loft in the Hill country 12-15 minutes directl...,This loft has a spectacular view of the hills ...,Loft in the Hill country 12-15 minutes directl...,none,This neighborhood is located in the hills west...,...,10.0,10.0,10.0,NaN,NaN,moderate,1.0,3.50,"30.3373609355,-97.8632766782","Host Is Superhost,Host Has Profile Pic,Host Id..."
1,17471131,https://www.airbnb.com/rooms/17471131,20170407214050,2017-04-08,"Claris I, Friendly Rentals","This apartment has: 1 double bed, 1 double sof...","This apartment has: 1 double bed, 1 double sof...","This apartment has: 1 double bed, 1 double sof...",none,Plaza Cataluña Catalonia Square is the large...,...,10.0,8.0,10.0,HUTB-006721,NaN,super_strict_30,106.0,0.86,"41.3896829422,2.17262543017","Host Has Profile Pic,Requires License,Instant ..."
2,17584891,https://www.airbnb.com/rooms/17584891,20170407214050,2017-04-08,"Style Terrace Red, Friendly Rentals","This apartment has: 1 double bed, 1 double sof...","This apartment has: 1 double bed, 1 double sof...","This apartment has: 1 double bed, 1 double sof...",none,Plaza Cataluña Catalonia Square is the large...,...,NaN,NaN,NaN,HUTB-007527,NaN,super_strict_30,106.0,NaN,"41.3930345489,2.16217327868","Host Has Profile Pic,Requires License,Instant ..."
3,5398030,https://www.airbnb.com/rooms/5398030,20170407214050,2017-04-08,Picasso Suite 1.4 Paseo de Gracia,Live like a local in Barcelona's most chic dis...,You wake up to the sun rising over Barcelona's...,Live like a local in Barcelona's most chic dis...,none,We're right in the heart of the chic L'Eixampl...,...,10.0,10.0,9.0,NaN,NaN,strict,24.0,1.09,"41.3969668101,2.1674178103","Host Has Profile Pic,Host Identity Verified,Re..."
4,18104606,https://www.airbnb.com/rooms/18104606,20170407214050,2017-04-08,Smart City Centre Apartment II,"Unique apartment in vibrant neighborhoods, car...","License: HUTB-005313 Charming apartment, locat...","Unique apartment in vibrant neighborhoods, car...",none,NaN,...,NaN,NaN,NaN,NaN,NaN,flexible,92.0,NaN,"41.3886851936,2.15514963616","Host Has Profile Pic,Host Identity Verified,Is..."
5,15695269,https://www.airbnb.com/rooms/15695269,20170407214050,2017-04-08,55. Valencia II Apartment,"Unique apartment in vibrant neighborhoods, car...",This fully refurbished apartment has 2 bedroom...,"Unique apartment in vibrant neighborhoods, car...",none,NaN,...,NaN,NaN,NaN,NaN,NaN,strict,92.0,NaN,"41.3825366724,2.14957707323","Host Has Profile Pic,Host Identity Verified,Is..."
6,1295958,https://www.airbnb.com/rooms/1295958,20170407214050,2017-04-08,3 BR + 2 baths near the beach!!,Spacious and quiet apartment that can accommod...,Its size allows to accommodate up to 6 people ...,Spacious and quiet apartment that can accommod...,none,"This unique area is rich in history, architect...",...,10.0,9.0,9.0,HUTB-0077039-0,NaN,strict,1.0,1.22,"41.3990576554,2.18422942312","Host Has Profile Pic,Is Location Exact,Require..."


In [6]:
airbnb.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14780 entries, 0 to 14779
Data columns (total 89 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   ID                              14780 non-null  int64  
 1   Listing Url                     14780 non-null  object 
 2   Scrape ID                       14780 non-null  int64  
 3   Last Scraped                    14780 non-null  object 
 4   Name                            14779 non-null  object 
 5   Summary                         14189 non-null  object 
 6   Space                           10888 non-null  object 
 7   Description                     14774 non-null  object 
 8   Experiences Offered             14780 non-null  object 
 9   Neighborhood Overview           9134 non-null   object 
 10  Notes                           5644 non-null   object 
 11  Transit                         9066 non-null   object 
 12  Access                          

In [12]:
# Listado de variables que carecen de interés para el estudio
del_columns = ['Listing Url', 'Scrape ID', 'Last Scraped', 'Name', 'Summary', 'Space', 'Description', 'Experiences Offered', 'Neighborhood Overview', 'Notes', 'Transit', 'Access', 'Interaction', 'House Rules', 'Thumbnail Url', 'Medium Url', 'Picture Url', 'XL Picture Url', 'Host ID', 'Host URL', 'Host Name', 'Host Since', 'Host About', 'Host Acceptance Rate', 'Host Thumbnail Url', 'Host Picture Url', 'Host Verifications', 'Street', 'Neighbourhood', 'Neighbourhood Group Cleansed', 'City', 'State', 'Zipcode', 'Market', 'Smart Location', 'Country Code', 'Country', 'Weekly Price', 'Monthly Price', 'Calendar Updated', 'Has Availability', 'Calendar last Scraped', 'First Review', 'Last Review', 'License', 'Jurisdiction Names']

El criterio para decidir si una variable carece de interés es subjetivo pero por lo general han sido seleccionadas todas las columnas que:
- Contengan url's
- Campos descriptivos del que no se piense hacer un análisis de NLP (la idea es estimar el precio con la mayor objetividad posible y cada host añade descriptivos que pueden distorsionar la realidad)
- Id's secundarios (mantenemos en los datos la columna ID por si hubiese que recuperar algún dato)
- Fechas referentes al método de estracción de datos
- Nombres y cualidades del host
- Columnas referentes a la dirección (excluyendo longitud y latitud)
- Fechas de reseñas
- Columnas con todos los datos missing (aun no se ha realizado ninguna función pero en el data wrangler usado como visor aparece por defecto el número total de valores faltantes y distintos)
- Otras columnas de Precios que son agregados del precio diario y presentaran a priori una alta correlación (presentaban muchos valores nulos)
- Número de Licencia y nombre de Jurisdicción

In [13]:
airbnb_col_fil = airbnb.drop(del_columns, axis =1)

In [23]:
print("Primer df:  ", airbnb.shape, "\n" + "Segundo df: ", airbnb_col_fil.shape)
# Hemos reducido más de la mitad del número de columnas (46)

Primer df:   (14780, 89) 
Segundo df:  (14780, 43)


Para mitigar cualquier selección de las muestras train y test con observaciones cuya variable de estudio sea nula, filtramos el dataframe. No queremos observaciones que no estén etiquetadas ya que los métodos que usaremos en el futuro requieren de dicha información para poder medir el error.

In [24]:
airbnb_row_fil = airbnb_col_fil[~airbnb_col_fil['Price'].isnull()]

In [25]:
print("Primer df:  ", airbnb_col_fil.shape, "\n" + "Segundo df: ", airbnb_row_fil.shape)
# Hemos eliminado 7 observaciones carentes de Precio

Primer df:   (14780, 43) 
Segundo df:  (14763, 43)


Ahora ya podemos dividir nuestra muestra de Airbnb en 2 sub muestras train con la que entrenaremos el modelo y test con la que evaluaremos como desempeña

In [39]:
import sklearn
from sklearn.model_selection import train_test_split

train, test = train_test_split(airbnb_row_fil, test_size = 0.3, shuffle = True, random_state = 420)

print(f'Dimensiones del dataset de training: {train.shape}, proporción de {(train.shape[0]/airbnb_row_fil.shape[0])*100:.0f}%')
print(f'Dimensiones del dataset de test: {test.shape}, proporción de {(test.shape[0]/airbnb_row_fil.shape[0])*100:.0f}%')

# Guardamos ambos ficheros aunque hemos especificado una semilla en el random_state
train.to_csv(r'data/airbnb_train.csv', sep=';', decimal='.', index=False)
test.to_csv(r'data/airbnb_test.csv', sep=';', decimal='.', index=False)

Dimensiones del dataset de training: (10334, 43), proporción de 70%
Dimensiones del dataset de test: (4429, 43), proporción de 30%


#### 2. Análisis Exploratorio